In [ ]:
%matplotlib inline

from chainer import cuda, Variable, FunctionSet, optimizers
import chainer.functions as F
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np

mlp.style.use('ggplot')

# Deep Lerning - 手書き文字認識

以下の手順で進みます

- データの確認
- Chainerを使ってDeep Learning
  - 活性化関数
  - 情報の伝搬処理
  - 各ユニットの初期値

参考：[ニューラルネットワークで数字を認識するWebアプリを作る](http://qiita.com/ginrou@github/items/07b52a8520efcaebce37)

## データの確認

- 入力層のサイズ：784次元 (画像サイズ：28x28)
- 出力層のサイズ：10次元 (0-9)
- 中間層のサイズ：300次元

In [ ]:
mnist = np.load('data/mnist.npz')
x_train = mnist['x_train'].astype(np.float32)
x_test = mnist['x_test'].astype(np.float32)
y_train = mnist['y_train'].astype(np.int32)
y_test = mnist['y_test'].astype(np.int32)

In [ ]:
# 訓練データのサイズ
x_train.shape

In [ ]:
# テストデータのサイズ
x_test.shape

In [ ]:
# データの中身はどうなっているのか？
print(x_train[25000][:90])
print(y_train[25000])

In [ ]:
def draw_digit(data):
    size = 28
    X, Y = np.meshgrid(range(size), range(size))
    Z = data.reshape(size, size)
    Z = Z[::-1,:]

    plt.figure(figsize=(2.5, 2.5))
    
    plt.xlim(0, 27)
    plt.ylim(0, 27)
    plt.pcolor(X, Y, Z)
    plt.gray()
    plt.tick_params(labelbottom="off")
    plt.tick_params(labelleft="off")
    
    plt.show()

# 個々のデータを描画してみる

draw_digit(x_train[0])
draw_digit(x_train[40000])
draw_digit(x_train[-1])

## Chainerを使ってDeep Learning

- 活性化関数
- 情報の伝搬処理
- 重みの更新 ( ※時間的な問題で今回説明しません )

### 活性化関数

In [ ]:
# 関数の形を描画してみる

x_data = np.linspace(-10, 10, 100, dtype=np.float32)
x = Variable(x_data)
y = F.sigmoid(x)

plt.plot(x.data, y.data, 'o--')

In [ ]:
# サンプルだと Rectified Linear Unit function が使われている
x_data = np.linspace(-10, 10, 100, dtype=np.float32)
x = Variable(x_data)
y = F.relu(x)

plt.plot(x.data, y.data, 'o--')

### モデルの作成

In [ ]:
def forward(x_data, y_data, train=True):
    x, t = Variable(x_data), Variable(y_data)
    h1 = F.dropout(F.relu(model.l1(x)),  train=train)
    h2 = F.dropout(F.relu(model.l2(h1)), train=train)
    y  = model.l3(h2)
    return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

def batch(train_size, batchsize):
    sum_accuracy = 0
    sum_loss     = 0
    
    for i in xrange(0, train_size, batchsize):
        x_batch = x_train[perm[i:i+batchsize]]
        y_batch = y_train[perm[i:i+batchsize]]

        optimizer.zero_grads()
        loss, acc = forward(x_batch, y_batch)
        loss.backward()
        optimizer.update()

        sum_loss     += float(cuda.to_cpu(loss.data)) * batchsize
        sum_accuracy += float(cuda.to_cpu(acc.data)) * batchsize

    return sum_loss, sum_accuracy

### 評価

In [ ]:
def evaluate(test_size, batchsize):
    sum_accuracy = 0
    sum_loss     = 0
    for i in xrange(0, test_size, batchsize):
        x_batch = x_test[i:i+batchsize]
        y_batch = y_test[i:i+batchsize]

        loss, acc = forward(x_batch, y_batch, train=False)

        sum_loss     += float(cuda.to_cpu(loss.data)) * batchsize
        sum_accuracy += float(cuda.to_cpu(acc.data)) * batchsize

    return sum_loss, sum_accuracy

## 作成したDeep Learningのモデルで実験

作成したDeep Learningのモデルを使って文字認識の実験をしてみましょう

- 学習
- 予測

In [ ]:
train_loss = []
train_acc  = []
test_loss = []
test_acc  = []

l1_W = []
l2_W = []
l3_W = []

input_size = 784
output_size = 10
hidden_size = 300

model = FunctionSet(l1=F.Linear(input_size, hidden_size),
                    l2=F.Linear(hidden_size, hidden_size),
                    l3=F.Linear(hidden_size, output_size))
optimizer = optimizers.Adam()
optimizer.setup(model.collect_parameters())

batchsize = 100  # 確率的勾配降下法で学習させる際の１回分のバッチサイズ
n_epoch   = 20  # 学習の繰り返し回数
train_size = x_train.shape[0]
test_size = x_test.shape[0]

for epoch in xrange(1, n_epoch+1):
    print('epoch: {0}'.format(epoch))

    perm = np.random.permutation(train_size)

    sum_loss, sum_accuracy = batch(train_size, batchsize)
    print 'train mean loss={}, accuracy={}'.format(sum_loss / train_size, sum_accuracy / train_size)
    
    # テストデータでの誤差と、正解精度を表示
    sum_loss, sum_accuracy = evaluate(test_size, batchsize)
    print 'test  mean loss={}, accuracy={}'.format(sum_loss / test_size, sum_accuracy / test_size)

    # 学習したパラメーターを保存
    l1_W.append(model.l1.W)
    l2_W.append(model.l2.W)
    l3_W.append(model.l3.W)

## 作ったモデルを確認してみる

In [ ]:
plt.style.use('fivethirtyeight')
def draw_digit3(data, n, ans, recog):
    size = 28
    plt.subplot(10, 10, n)
    Z = data.reshape(size,size)   # convert from vector to 28x28 matrix
    Z = Z[::-1,:]             # flip vertical
    plt.xlim(0,27)
    plt.ylim(0,27)
    plt.pcolor(Z)
    plt.title("ans=%d, recog=%d"%(ans,recog), size=8)
    plt.gray()
    plt.tick_params(labelbottom="off")
    plt.tick_params(labelleft="off")


plt.figure(figsize=(15,15))

cnt = 0
for idx in np.random.permutation(train_size)[:100]:

    xxx = x_train[idx].astype(np.float32)
    h1 = F.dropout(F.relu(model.l1(Variable(xxx.reshape(1,784)))),  train=False)
    h2 = F.dropout(F.relu(model.l2(h1)), train=False)
    y  = model.l3(h2)
    cnt+=1
    draw_digit3(x_train[idx], cnt, y_train[idx], np.argmax(y.data))
plt.show()

In [ ]:
def draw_digit2(data, n, i):
    size = 28
    plt.subplot(10, 10, n)
    Z = data.reshape(size,size)   # convert from vector to 28x28 matrix
    Z = Z[::-1,:]             # flip vertical
    plt.xlim(0,27)
    plt.ylim(0,27)
    plt.pcolor(Z)
    plt.title("%d"%i, size=9)
    plt.gray()
    plt.tick_params(labelbottom="off")
    plt.tick_params(labelleft="off")

plt.figure(figsize=(10,10))
cnt = 1
for i in np.random.permutation(hidden_size)[:100]:
    draw_digit2(l1_W[len(l1_W)-1][i], cnt, i)
    cnt += 1

plt.show()